# 安裝並導入所需的套件

In [ ]:
!pip install langchain langchain-openai langchain-community chromadb pypdf

In [ ]:
# 導入必要的模組
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

# 設定 OpenAI API Key（請替換為您的實際 API Key）
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

## 第一步：知識庫建設階段

### 步驟ㄧ、載入文件
這裡我們創建一個示例文件來演示

In [ ]:
sample_text = """
公司遠端工作政策

為了提供員工更大的工作靈活性，本公司制定以下遠端工作政策：

1. 遠端工作資格
- 所有全職員工在入職滿 6 個月後，可申請遠端工作
- 部分職位可能因工作性質不適用遠端工作

2. 工作時間要求
- 遠端工作者須維持標準工作時間：上午 9 點至下午 6 點
- 必須在團隊核心會議時間（上午 10 點至下午 4 點）保持在線

3. 設備與技術要求
- 公司提供筆記型電腦和必要軟體
- 員工需確保穩定的網路連線
- 須安裝公司指定的安全軟體

4. 績效評估
- 遠端工作員工的績效評估標準與辦公室員工相同
- 採用目標導向的評估方式
"""


In [ ]:
# 將示例文字儲存為檔案
with open("company_policy.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

# 使用 TextLoader 載入文件
loader = TextLoader("company_policy.txt", encoding="utf-8")
documents = loader.load()

print(f"成功載入 {len(documents)} 個文件")
print(f"文件內容長度：{len(documents[0].page_content)} 字符")

### 步驟二、文件分割

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # 每個片段的最大字符數
    chunk_overlap=50,      # 片段之間的重疊字符數
    length_function=len,   # 使用字符數作為長度計算
    separators=["\n\n", "\n", " ", ""]  # 分割的優先順序
)

# 執行分割
text_chunks = text_splitter.split_documents(documents)

print(f"文件已分割為 {len(text_chunks)} 個片段")
for i, chunk in enumerate(text_chunks):
    print(f"片段 {i+1} 長度：{len(chunk.page_content)} 字符")
    print(f"內容預覽：{chunk.page_content[:100]}...")
    print("-" * 50)

### 步驟三、生成嵌入並建立向量資料庫

In [ ]:
embeddings = OpenAIEmbeddings()

# 建立向量資料庫
vector_store = Chroma.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"  # 本地持久化儲存
)

print("向量資料庫建立完成！")
print(f"資料庫中共有 {vector_store._collection.count()} 個向量")


## 第二步、查詢處理階段

### 步驟ㄧ、設定檢索器

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",    # 使用相似度搜尋
    search_kwargs={"k": 3}      # 檢索最相關的 3 個片段
)

# 測試檢索功能
user_question = "公司的遠端工作政策是什麼？"
relevant_docs = retriever.get_relevant_documents(user_question)

print(f"問題：{user_question}")
print(f"找到 {len(relevant_docs)} 個相關文件片段：\n")

for i, doc in enumerate(relevant_docs):
    print(f"片段 {i+1}:")
    print(doc.page_content)
    print("-" * 50)


### 步驟二、設定語言模型和提示模板

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1  # 較低的溫度確保回答的一致性
)

# 自訂提示模板
prompt_template = """你是一個專業的企業政策諮詢助理。請根據以下提供的公司文件內容，回答用戶的問題。

相關文件內容：
{context}

用戶問題：{question}

請注意：
1. 請只基於提供的文件內容回答，不要添加文件中沒有的資訊
2. 如果文件中沒有相關資訊，請明確說明
3. 回答要清晰、準確且有條理

回答："""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

### 步驟三、建立完整 RAG 鏈

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",           # 將所有檢索到的文件合併後傳給 LLM
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True  # 同時返回來源文件
)

# 執行查詢
result = qa_chain({"query": user_question})

print("=== RAG 系統回答 ===")
print(f"問題：{result['query']}")
print(f"答案：{result['result']}")
print("\n=== 來源文件 ===")
for i, doc in enumerate(result['source_documents']):
    print(f"來源 {i+1}：{doc.page_content[:200]}...")


測試問題

In [ ]:
# 測試更多問題
test_questions = [
    "遠端工作需要滿足什麼資格？",
    "公司提供哪些設備給遠端工作者？",
    "遠端工作者的工作時間要求是什麼？",
    "如何評估遠端工作員工的績效？"
]

for question in test_questions:
    print(f"\n問題：{question}")
    result = qa_chain({"query": question})
    print(f"答案：{result['result']}")
    print("-" * 80)
